In [ ]:
# Import Library
from operator import index
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import os
import codecs, json
import tempfile
import requests
import base64
import csv
import numpy as np
np.set_printoptions(precision=4)
import seaborn as sns
sns.set(style='whitegrid', font_scale=1.3, color_codes=True)
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling
# from pivottablejs import pivot_ui
import webbrowser
import plotly.express as px
from bokeh.plotting import Figure, figure, show, output_notebook,output_file
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, HoverTool
from bokeh.palettes import Spectral4
from bokeh.io import push_notebook
from bokeh.models import Range1d
output_notebook()
from ipywidgets import interact

In [ ]:
# Read Data
from google.colab import drive
drive.mount("/content/drive")
df = pd.read_csv('drive/MyDrive/Project/AI/EPL/results.csv', low_memory=False, encoding='latin1')
df.info()
df.sample(5)

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Season    11113 non-null  object 
 1   DateTime  11113 non-null  object 
 2   HomeTeam  11113 non-null  object 
 3   AwayTeam  11113 non-null  object 
 4   FTHG      11113 non-null  int64  
 5   FTAG      11113 non-null  int64  
 6   FTR       11113 non-null  object 
 7   HTHG      10189 non-null  float64
 8   HTAG      10189 non-null  float64
 9   HTR       10189 non-null  object 
 10  Referee   8289 non-null   object 
 11  HS        8289 non-null   float64
 12  AS        8289 non-null   float64
 13  HST       8289 non-null   float64
 14  AST       8289 non-null   float64
 15  HC        8289 non-null   float64
 16  AC        8289 non-null   float64
 17  HF        8289 non-null   float64
 18  AF        8289 non-null   float64
 19  HY        8289 non-null   float64
 20  AY

,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
3779,2002-03,2002-12-26T00:00:00Z,Sunderland,Leeds,1,2,A,1.0,0.0,H,...,6.0,4.0,8.0,3.0,12.0,11.0,0.0,1.0,0.0,0.0
9524,2017-18,2018-01-30T00:00:00Z,Huddersfield,Liverpool,0,3,A,0.0,2.0,A,...,1.0,7.0,1.0,4.0,8.0,8.0,0.0,0.0,0.0,0.0
3593,2002-03,2002-08-19T00:00:00Z,Newcastle,West Ham,4,0,H,0.0,0.0,D,...,10.0,4.0,7.0,1.0,7.0,9.0,1.0,2.0,0.0,0.0
1164,1995-96,1996-02-03T00:00:00Z,Blackburn,Bolton,3,1,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5412,2006-07,2007-04-02T00:00:00Z,Aston Villa,Everton,1,1,D,0.0,1.0,A,...,10.0,8.0,5.0,6.0,18.0,16.0,3.0,3.0,0.0,0.0


In [ ]:
# Convert Date into Day
df["matchDate"] = pd.to_datetime(df["DateTime"], infer_datetime_format=True)
df['matchDay'] = df['matchDate'].dt.day_name()
print(df["matchDate"][0])
print(df['matchDay'][149])
df.drop('DateTime', axis=1, inplace=True)
df.drop('matchDate', axis=1, inplace=True)
df.head()

df.matchDay.value_counts()

1993-08-14 00:00:00
Saturday


Saturday     6394
Sunday       2185
Wednesday     993
Monday        738
Tuesday       558
Friday        129
Thursday      116
Name: matchDay, dtype: int64

In [ ]:
# Function to visualize the features with missing values
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_type = df.dtypes
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_type], axis = 1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {
        0 : 'Missing Values', 1 : '% of Total Values', 2: 'type'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    print ('Your selected dataframe has ' + str(df.shape[1]) + ' columns.\n' 'There are ' + str(
        mis_val_table_ren_columns.shape[0]) + ' columns that have missing values.')
    return mis_val_table_ren_columns

missing_values_table(df)

Your selected dataframe has 23 columns.
There are 16 columns that have missing values.


,Missing Values,% of Total Values,type
Referee,2824,25.4,object
HS,2824,25.4,float64
AS,2824,25.4,float64
HST,2824,25.4,float64
AST,2824,25.4,float64
HC,2824,25.4,float64
AC,2824,25.4,float64
HF,2824,25.4,float64
AF,2824,25.4,float64
HY,2824,25.4,float64


In [ ]:
# drop missing row
df = df.dropna().reset_index(drop=True)

# drop referee
df.drop('Referee', axis=1, inplace=True)

missing_values_table(df)

df.Season.value_counts()

Your selected dataframe has 22 columns.
There are 0 columns that have missing values.


2000-01    380
2001-02    380
2020-21    380
2019-20    380
2018-19    380
2017-18    380
2016-17    380
2015-16    380
2014-15    380
2013-14    380
2012-13    380
2011-12    380
2010-11    380
2009-10    380
2008-09    380
2007-08    380
2006-07    380
2005-06    380
2004-05    380
2003-04    380
2002-03    380
2021-22    309
Name: Season, dtype: int64

In [ ]:
# convert Season to num
df['Season'] = df['Season'].str[-2:].astype('int')

# convert FTR and HTR to int
df['FTR'] = df['FTR'].map(
    {'H': 1, 'A': -1, 'D': 0})

df['HTR'] = df['HTR'].map(
    {'H': 1, 'A': -1, 'D': 0})

In [ ]:
# add feature comeback
def comeback(HT, FT):
    if HT == -1 and FT == 1:
        return 1
    elif HT == 1 and FT == -1:
        return -1
    else:
        return 0
df['comeback'] = df.apply(lambda row: comeback(row.HTR, row.FTR), axis = 1)
df['comeback2']=np.where((df['FTR']!=df['HTR'])&(df['FTR']!='D') & (df['HTR']!='D'),1,0)


In [ ]:
# split dftrain and testing
df_train = df[df.Season <21]
df_test = df[df.Season == 21]

In [ ]:
## CREATE HAS HAC AAS AAC
home_stat = df_train.groupby('HomeTeam').mean()
away_stat = df_train.groupby('AwayTeam').mean()

home_stat['HAS'] = home_stat['FTHG']
home_stat['HAC'] = home_stat['FTAG']
away_stat['AAS'] = away_stat['FTAG']
away_stat['AAC'] = away_stat['FTHG']

team_stat = pd.DataFrame()
team_stat['Team'] = home_stat.index
team_stat

team_stat = team_stat.merge(home_stat['HAS'], left_on='Team', right_index=True)
team_stat = team_stat.merge(home_stat['HAC'], left_on='Team', right_index=True)

team_stat = team_stat.merge(away_stat['AAS'], left_on='Team', right_index=True)
team_stat = team_stat.merge(away_stat['AAC'], left_on='Team', right_index=True)

from sklearn.preprocessing import MinMaxScaler
team_stat.set_index("Team", inplace=True)
scaler = MinMaxScaler()
team_stat = pd.DataFrame(scaler.fit_transform(team_stat), columns=team_stat.columns, index=team_stat.index)

col_list = ['HomeTeam', 'AwayTeam', 'FTR', 'matchDay', 'comeback','Season']
new_df = df[col_list]

In [ ]:
## CREATE HWP HLP HCP AWP ALP ACP
home_win=round((df_train.loc[df_train['FTR']==1].groupby('HomeTeam')['FTR'].count()/df_train.groupby('HomeTeam')['FTR'].count()),3)
home_loss=round((df_train.loc[df_train['FTR']==-1].groupby('HomeTeam')['FTR'].count()/df_train.groupby('HomeTeam')['FTR'].count()),3)
home_draw=round((df_train.loc[df_train['FTR']==0].groupby('HomeTeam')['FTR'].count()/df_train.groupby('HomeTeam')['FTR'].count()),3)
home_cb = df_train.loc[(df_train['comeback']==1)].groupby('HomeTeam')['comeback2'].sum()
away_win=round((df_train.loc[df_train['FTR']==-1].groupby('AwayTeam')['FTR'].count()/df_train.groupby('AwayTeam')['FTR'].count()),3)
away_loss=round((df_train.loc[df_train['FTR']==1].groupby('AwayTeam')['FTR'].count()/df_train.groupby('AwayTeam')['FTR'].count()),3)
away_draw=round((df_train.loc[df_train['FTR']==0].groupby('AwayTeam')['FTR'].count()/df_train.groupby('AwayTeam')['FTR'].count()),3)
away_cb = df_train.loc[(df_train['comeback']==-1)].groupby('AwayTeam')['comeback2'].sum()

home_stat['HWP'] = home_win
home_stat['HLP'] = home_loss
home_stat['HDP'] = home_draw
home_stat['HCP'] = home_cb
away_stat['AWP'] = away_win
away_stat['ALP'] = away_loss
away_stat['ADP'] = away_draw
away_stat['ACP'] = away_cb

team_stat = team_stat.merge(home_stat['HWP'], left_on='Team', right_index=True)
team_stat = team_stat.merge(home_stat['HLP'], left_on='Team', right_index=True)
team_stat = team_stat.merge(home_stat['HDP'], left_on='Team', right_index=True)
team_stat = team_stat.merge(home_stat['HCP'], left_on='Team', right_index=True)

team_stat = team_stat.merge(away_stat['AWP'], left_on='Team', right_index=True)
team_stat = team_stat.merge(away_stat['ALP'], left_on='Team', right_index=True)
team_stat = team_stat.merge(away_stat['ADP'], left_on='Team', right_index=True)
team_stat = team_stat.merge(away_stat['ACP'], left_on='Team', right_index=True)


In [ ]:
# Missing Values: 10 in ACP, 6 in ACP. Replace with 0
team_stat = team_stat.fillna(0)

In [ ]:
# Model Predicting
# newdf
new_df = new_df.merge(team_stat['HAS'], left_on='HomeTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['HAC'], left_on='HomeTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['AAS'], left_on='AwayTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['AAC'], left_on='AwayTeam', right_index=True, how='left')

new_df['Hstr'] = (new_df['HAS'] * new_df['AAC']); # Home Attack Strength
new_df['Astr'] = (new_df['HAC'] * new_df['AAS']); # Away Attack Strength

new_df = new_df.merge(team_stat['HWP'], left_on='HomeTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['HLP'], left_on='HomeTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['HDP'], left_on='HomeTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['HCP'], left_on='HomeTeam', right_index=True, how='left')

new_df = new_df.merge(team_stat['AWP'], left_on='AwayTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['ALP'], left_on='AwayTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['ADP'], left_on='AwayTeam', right_index=True, how='left')
new_df = new_df.merge(team_stat['ACP'], left_on='AwayTeam', right_index=True, how='left')


In [ ]:
# Brentford isi 0
new_df = new_df.fillna(0)

In [ ]:
# SPLIT Training dan Testing
new_df = pd.get_dummies(new_df, columns=['matchDay'], prefix = ['matchDay'])

new_df_train = new_df[new_df.Season < 21]
new_df_test = new_df[new_df.Season == 21]

X_train = new_df_train.drop(['HomeTeam','AwayTeam','FTR'], axis=1)
y_train = new_df_train['FTR']
X_test = new_df_test.drop(['HomeTeam','AwayTeam','FTR'], axis=1)
y_test = new_df_test['FTR']


In [ ]:
# 1. Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc.score(X_train, y_train)

rfc_pred = rfc.predict(X_test)
rfc_pred

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
acc_score = accuracy_score(y_test, rfc_pred)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(rfc, X_train, y_train, cv=10)
print("Cross val scores")
print(scores)

score_train = rfc.score(X_train, y_train)
score_test = rfc.score(X_test, y_test)

print("Training set accuracy: ", '%.6f'%(score_train))
print("Test set accuracy: ", '%.6f'%(score_test))

Accuracy score: 0.44473684210526315
Cross val scores
[0.4368 0.4263 0.4803 0.4408 0.4421 0.4658 0.4539 0.4579 0.4592 0.4908]
Training set accuracy:  1.000000
Test set accuracy:  0.444737


In [ ]:
# 2. MLP
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes = (50, 50, 50),
                    random_state = 5,
                    verbose = False,
                    learning_rate_init = 0.01)

clf.fit(X_train, y_train)
clf_pred = clf.predict(X_test)

acc_score = accuracy_score(y_test, clf_pred)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Cross val scores")
print(scores)

score_train = clf.score(X_train, y_train)
score_test = clf.score(X_test, y_test)

print("Training set accuracy: ", '%.6f'%(score_train))
print("Test set accuracy: ", '%.6f'%(score_test))

Accuracy score: 0.5368421052631579
Cross val scores
[0.3987 0.5171 0.5118 0.5447 0.5592 0.5237 0.5487 0.5263 0.5487 0.5276]
Training set accuracy:  0.550395
Test set accuracy:  0.536842


In [ ]:
# 3. MLP2
mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(15,10,5),(5,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
from sklearn.model_selection import GridSearchCV
clf2 = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf2.fit(X_train, y_train)
clf2_pred = clf2.predict(X_test)

acc_score = accuracy_score(y_test, clf2_pred)
print("Accuracy score: " + str(acc_score))
# scores = cross_val_score(clf2, X_train, y_train, cv=10)
# print("Cross val scores")
# print(scores)

score_train = clf2.score(X_train, y_train)
score_test = clf2.score(X_test, y_test)

print("Training set accuracy: ", '%.6f'%(score_train))
print("Test set accuracy: ", '%.6f'%(score_test))

Accuracy score: 0.5157894736842106
Training set accuracy:  0.543947
Test set accuracy:  0.515789


In [ ]:
# 4. TF
y_train_tf = y_train
y_test_tf = y_test

y_train_tf = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_tf = tf.keras.utils.to_categorical(y_test, num_classes=3)
ctf = tf.keras.models.Sequential([
      tf.keras.layers.Dense(69, input_dim=23, activation='relu'),
      tf.keras.layers.Dense(12, input_dim=69, activation='relu'),
      tf.keras.layers.Dense(3, activation='softmax')
])

ctf.summary()
ctf.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ctf.fit(X_train, y_train_tf, epochs=100)
ctf_pred = ctf.predict(X_test)
ctf_pred_class = np.argmax(ctf_pred, axis=1)
ctf_pred_class[ctf_pred_class == 2] = -1

acc_score = accuracy_score(y_test, ctf_pred_class)
print("Accuracy score: " + str(acc_score))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 69)                1656      
                                                                 
 dense_1 (Dense)             (None, 12)                840       
                                                                 
 dense_2 (Dense)             (None, 3)                 39        
                                                                 
Total params: 2,535
Trainable params: 2,535
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
238/238 [==============================] - 1s 2ms/step - loss: 1.0400 - accuracy: 0.4971
Epoch 2/100
238/238 [==============================] - 0s 2ms/step - loss: 0.9902 - accuracy: 0.5196
Epoch 3/100
238/238 [==============================] - 0s 2ms/step - loss: 0.9690 - accuracy: 0.5343
Epoch 4/100
238

In [ ]:
##########
# Calculate Actual Points
epl21 = pd.DataFrame(index=team_stat.index)
epl21['Actual Points'] = 0
def make_actual_table(x, ht, at, which):
    if x == 1:
        epl21.loc[ht, which] += 3
    elif x == 0:
        epl21.loc[ht, which] += 1
        epl21.loc[at, which] += 1
    elif x == -1:
        epl21.loc[at, which] += 3
new_df_test.apply(lambda x: make_actual_table(
    x.FTR, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)

7600    None
7601    None
7602    None
7603    None
7604    None
        ... 
7975    None
7976    None
7977    None
7978    None
7979    None
Length: 380, dtype: object

In [ ]:
# Calculate Predicted RFC Points
df_rfc = pd.DataFrame(rfc_pred, columns=['Pred_RFC'])
new_df_test = new_df_test.reset_index(drop=True)
new_df_test = new_df_test.merge(df_rfc, left_index=True, right_index=True)

epl21['Predicted RFC'] = 0
new_df_test.apply(lambda x: make_actual_table(
    x.Pred_RFC, x.HomeTeam, x.AwayTeam, 'Predicted RFC'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [ ]:
# Calculate Predicted MLP Points
df_mlp = pd.DataFrame(clf_pred, columns=['Pred_MLP'])
new_df_test = new_df_test.reset_index(drop=True)
new_df_test = new_df_test.merge(df_mlp, left_index=True, right_index=True)

epl21['Predicted MLP'] = 0
new_df_test.apply(lambda x: make_actual_table(
    x.Pred_MLP, x.HomeTeam, x.AwayTeam, 'Predicted MLP'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [ ]:
# Calculate Predicted MLP2 Points
df_mlp2 = pd.DataFrame(clf_pred, columns=['Pred_MLP2'])
new_df_test = new_df_test.reset_index(drop=True)
new_df_test = new_df_test.merge(df_mlp2, left_index=True, right_index=True)

epl21['Predicted MLP2'] = 0
new_df_test.apply(lambda x: make_actual_table(
    x.Pred_MLP2, x.HomeTeam, x.AwayTeam, 'Predicted MLP2'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [ ]:
# Calculate Predicted TF Points
df_tf = pd.DataFrame(ctf_pred_class, columns=['Pred_TF'])
new_df_test = new_df_test.reset_index(drop=True)
new_df_test = new_df_test.merge(df_tf, left_index=True, right_index=True)

epl21['Predicted TF'] = 0
new_df_test.apply(lambda x: make_actual_table(
    x.Pred_TF, x.HomeTeam, x.AwayTeam, 'Predicted TF'), axis=1)

### END ###

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [ ]:
epl21.sort_values(by=['Actual Points'], ascending=False)

,Actual Points,Predicted RFC,Predicted MLP,Predicted MLP2,Predicted TF
Team,,,,,
Man City,86,96,87,87,102
Man United,74,81,108,108,99
Liverpool,69,99,102,102,94
Chelsea,67,75,111,111,97
Leicester,66,53,60,60,64
West Ham,65,30,36,36,26
Tottenham,62,68,87,87,83
Arsenal,61,61,99,99,94
Everton,59,54,63,63,60
